In [1]:

import tensorflow as tf
from tensorflow import keras

train_dataset_path = "../dataset/dog-breed-imagefolder/train"
training_dataset, validation_dataset = tf.keras.utils.image_dataset_from_directory(train_dataset_path, 
                                            image_size = (224, 224),  # Default value is (256, 256). **source of potential error**
                                            validation_split = 0.1, subset = "both", seed = 10, # Need to set subset & seed both for validation_split
                                            batch_size = 32, shuffle= True, # Default values used automatically. **SOURCE OF POTENTIAL ERROR**
                                            color_mode = "rgb", # Channels = 3. Hidden from us
                                            label_mode = "categorical" ) # MUST. OTHERWISE causes error at LOSS value calculation. Need to do one hot encoding there


Found 10222 files belonging to 120 classes.
Using 9200 files for training.
Using 1022 files for validation.


"\n    !IMP: 4 Bugs in the code\n    1. for directory, gave path. dog-breed-imagefolder. It lead to 2 classes train & test, instead of 120 classes as dog breeds\n    2. label_mode=int. it doesn't convert class to vector. categorical loss function needs vector of class not int. categorical_cross_entropy vs sparse_categorical_cross_entropy\n    3. validation_split needs subset & seed. Tensorflow requirements\n    4. default batch_size is 32. shuffle= True. These default values can lead to confusion while debugging with single element batch.\n    \n"


**IMP: 4 Bugs in the code**
1. for directory, gave path. dog-breed-imagefolder. It lead to 2 classes train & test, instead of 120 classes as dog breeds
2. label_mode=int. it doesn't convert class to vector. categorical loss function needs vector of class not int. categorical_cross_entropy vs sparse_categorical_cross_entropy. needed to write custom training loop because of this reason.
3. validation_split needs subset & seed. Tensorflow requirements
4. default batch_size is 32. shuffle= True. These default values can lead to confusion while debugging with single element batch.


In [2]:
NUM_CLASSES = 120

class simple_fcnn(keras.Model):
    def __init__(self):
        super().__init__()

        self.internal_model = tf.keras.models.Sequential([
            keras.layers.Input(shape = (224, 224, 3)),
            keras.layers.Flatten(),
            keras.layers.Dense(units = 240, activation="relu", ),
            keras.layers.Dense(units = NUM_CLASSES),
        ])

    def call(self, input_single_batch):
        final_layer_output = self.internal_model(input_single_batch)

        return final_layer_output

model  = simple_fcnn()
model.compile(
    loss      = keras.losses.CategoricalCrossentropy(),
    optimizer = keras.optimizers.Adam() 
)

In [ ]:
model.fit(x = training_dataset, validation_data = validation_dataset)